# retriving the data from chembl dataset

In [10]:
import pandas as pd
from chembl_webresource_client.new_client import new_client
from tqdm import tqdm
import time

def fetch_ache_ligands():
    target_client = new_client.target
    activity_client = new_client.activity
    
    print("Searching for Acetylcholinesterase (AChE) target...")
    target = target_client.search('P22303')
    if not target:
        print("No target found!")
        return None
    
    target_id = target[0]['target_chembl_id']
    print(f"Found target: {target_id}")
    
    print("Fetching ligand activities...")
    activities = []
    offset = 0
    chunk_size = 2000  # Increased chunk size for faster retrieval
    
    while True:
        try:
            chunk = list(activity_client.filter(target_chembl_id=target_id).only(
                ['molecule_chembl_id', 'standard_type', 'standard_value', 'standard_units', 'pchembl_value']
            )[offset:offset + chunk_size])
            
            if not chunk:
                break  # Stop when no more data is retrieved
            
            activities.extend(chunk)
            offset += len(chunk)  # Adjust offset dynamically
            print(f"Retrieved {len(activities)} activities so far... (offset: {offset})")
            time.sleep(0.5)  # Reduced delay to optimize retrieval speed
        except Exception as e:
            print(f"Error fetching activities at offset {offset}: {e}")
            break
    
    print(f"Total activities retrieved: {len(activities)}")
    df = pd.DataFrame(activities)
    print(f"Initial DataFrame shape: {df.shape}")
    
    # Keep only activity types relevant to binding
    df = df[df['standard_type'].isin(['IC50', 'Ki', 'Kd'])]
    print(f"Shape after filtering activity types: {df.shape}")
    
    # Convert numeric columns
    df['standard_value'] = pd.to_numeric(df['standard_value'], errors='coerce')
    df['pchembl_value'] = pd.to_numeric(df['pchembl_value'], errors='coerce')
    
    # Drop missing values
    df_cleaned = df.dropna(subset=['pchembl_value'])
    print(f"Shape after dropping missing pChEMBL values: {df_cleaned.shape}")
    
    # Fetch molecular structures (SMILES) with progress bar
    molecule_client = new_client.molecule
    smiles_data = []
    print("Fetching molecular structures...")
    for chembl_id in tqdm(df_cleaned['molecule_chembl_id'].unique(), desc="Fetching molecular structures"):
        try:
            mol = molecule_client.get(chembl_id)
            if 'molecule_structures' in mol and mol['molecule_structures']:
                smiles_data.append((chembl_id, mol['molecule_structures']['canonical_smiles']))
            else:
                print(f"No SMILES found for {chembl_id}")
        except Exception as e:
            print(f"Error fetching SMILES for {chembl_id}: {e}")
    
    # Convert to DataFrame
    smiles_df = pd.DataFrame(smiles_data, columns=['molecule_chembl_id', 'SMILES'])
    print(f"SMILES DataFrame shape: {smiles_df.shape}")
    
    df_final = df_cleaned.merge(smiles_df, on='molecule_chembl_id', how='left')
    print(f"Final merged DataFrame shape: {df_final.shape}")
    
    # Save to CSV
    df_final.to_csv('AChE_ligands.csv', index=False)
    print("Dataset saved as AChE_ligands.csv")
    return df_final

# Fetch and save dataset
dataset = fetch_ache_ligands()
if dataset is not None:
    print(dataset.head())

Searching for Acetylcholinesterase (AChE) target...
Found target: CHEMBL220
Fetching ligand activities...
Retrieved 2000 activities so far... (offset: 2000)
Retrieved 2020 activities so far... (offset: 2020)
Retrieved 2040 activities so far... (offset: 2040)
Retrieved 2060 activities so far... (offset: 2060)
Retrieved 2080 activities so far... (offset: 2080)
Retrieved 2100 activities so far... (offset: 2100)
Retrieved 2120 activities so far... (offset: 2120)
Retrieved 2140 activities so far... (offset: 2140)
Retrieved 2160 activities so far... (offset: 2160)
Retrieved 2180 activities so far... (offset: 2180)
Retrieved 2200 activities so far... (offset: 2200)
Retrieved 2220 activities so far... (offset: 2220)
Retrieved 2240 activities so far... (offset: 2240)
Retrieved 2260 activities so far... (offset: 2260)
Retrieved 2280 activities so far... (offset: 2280)
Retrieved 2300 activities so far... (offset: 2300)
Retrieved 2320 activities so far... (offset: 2320)
Retrieved 2340 activities s

Fetching molecular structures:  21%|█████████▍                                   | 1197/5703 [16:30<1:04:05,  1.17it/s]

No SMILES found for CHEMBL2448138


Fetching molecular structures:  80%|██████████████████████████████████▌        | 4585/5703 [1:02:11<4:21:42, 14.05s/it]

Error fetching SMILES for CHEMBL4793341: HTTPSConnectionPool(host='www.ebi.ac.uk', port=443): Max retries exceeded with url: /chembl/api/data/molecule/CHEMBL4793341 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001E4BC357490>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))


...

Error fetching SMILES for CHEMBL4760651: HTTPSConnectionPool(host='www.ebi.ac.uk', port=443): Max retries exceeded with url: /chembl/api/data/molecule/CHEMBL4760651 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001E4B450AB10>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))


Fetching molecular structures:  83%|███████████████████████████████████▊       | 4742/5703 [1:11:41<8:18:33, 31.13s/it]

Error fetching SMILES for CHEMBL4764003: HTTPSConnectionPool(host='www.ebi.ac.uk', port=443): Max retries exceeded with url: /chembl/api/data/molecule/CHEMBL4764003 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001E4B4524D50>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))


Fetching molecular structures: 100%|█████████████████████████████████████████████| 5703/5703 [1:30:31<00:00,  1.05it/s]

SMILES DataFrame shape: (5699, 2)
Final merged DataFrame shape: (7549, 9)
Dataset saved as AChE_ligands.csv
  molecule_chembl_id  pchembl_value standard_type standard_units  \
0       CHEMBL133897           6.12          IC50             nM   
1       CHEMBL336398           7.00          IC50             nM   
2       CHEMBL130628           6.52          IC50             nM   
3       CHEMBL130478           6.10          IC50             nM   
4       CHEMBL130112           5.62          IC50             nM   

   standard_value  type units value  \
0           750.0  IC50    uM  0.75   
1           100.0  IC50    uM   0.1   
2           300.0  IC50    uM   0.3   
3           800.0  IC50    uM   0.8   
4          2400.0  IC50    uM   2.4   

                                          SMILES  
0          CCOc1nn(-c2cccc(OCc3ccccc3)c2)c(=O)o1  
1     O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC1CC1  
2  O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F  
3      CSc1nc(-c2ccc(OC(F)(F)F)cc2)nn1C(=O)N

# removing unnecessary columns from the data

In [13]:
import pandas as pd

# Load the dataset
file_path = r"C:\Users\ngaga\OneDrive\Desktop\aragen\AChE_ligands_csv.csv"  # Update with actual path
df = pd.read_csv(file_path)

# Drop the unnecessary columns
columns_to_drop = ['type', 'units', 'value']
df.drop(columns=columns_to_drop, inplace=True, errors='ignore')  # Ignore errors if columns don't exist

# Save the cleaned dataset back to a CSV file
cleaned_file_path = r"C:\Users\ngaga\OneDrive\Desktop\aragen\AChE_ligands_cleaned.csv"
df.to_csv(cleaned_file_path, index=False)

print(f"Cleaned dataset saved as {cleaned_file_path}")
print(df.head())  # Preview the cleaned dataset


Cleaned dataset saved as C:\Users\ngaga\OneDrive\Desktop\aragen\AChE_ligands_cleaned.csv
  molecule_chembl_id  pchembl_value standard_type standard_units  \
0       CHEMBL133897           6.12          IC50             nM   
1       CHEMBL336398           7.00          IC50             nM   
2       CHEMBL130628           6.52          IC50             nM   
3       CHEMBL130478           6.10          IC50             nM   
4       CHEMBL130112           5.62          IC50             nM   

   standard_value                                         SMILES  
0           750.0          CCOc1nn(-c2cccc(OCc3ccccc3)c2)c(=O)o1  
1           100.0     O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC1CC1  
2           300.0  O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F  
3           800.0      CSc1nc(-c2ccc(OC(F)(F)F)cc2)nn1C(=O)N(C)C  
4          2400.0       CSc1nc(-c2ccc(C)cc2)nn1C(=O)N(C)c1ccccc1  


# determining if a ligand is active or inactive based on activity score(70%-pCHEMBL value + 30%-standard value)

In [14]:
import pandas as pd
import numpy as np

# Load the dataset
file_path = r"C:\Users\ngaga\OneDrive\Desktop\aragen\AChE_ligands_cleaned.csv"
df = pd.read_csv(file_path)

# Ensure numeric columns
df['pchembl_value'] = pd.to_numeric(df['pchembl_value'], errors='coerce')
df['standard_value'] = pd.to_numeric(df['standard_value'], errors='coerce')

# Normalization function for pChEMBL value (scaling between 0 and 1)
def normalize_pchembl(pchembl):
    return min(max((pchembl - 4) / 4, 0), 1) if not np.isnan(pchembl) else 0

# Normalization function for Standard Value (inverse scale, lower is better)
def normalize_standard_value(value):
    return min(max(1 - (value / 1000), 0), 1) if not np.isnan(value) else 0

# Compute Activity Score
df['pchembl_score'] = df['pchembl_value'].apply(normalize_pchembl)
df['standard_score'] = df.apply(lambda row: normalize_standard_value(row['standard_value']) 
                                if row['standard_type'] in ['IC50', 'Ki', 'Kd'] else 0, axis=1)

df['Activity_Score'] = 0.7 * df['pchembl_score'] + 0.3 * df['standard_score']

# Classify as Active or Inactive
df['Activity'] = df['Activity_Score'].apply(lambda x: 'Active' if x >= 0.5 else 'Inactive')

# Save the updated dataset
updated_file_path = r"C:\Users\ngaga\OneDrive\Desktop\aragen\AChE_ligands_weighted_activity.csv"
df.to_csv(updated_file_path, index=False)

print(f"Updated dataset saved as {updated_file_path}")
print(df['Activity'].value_counts())  # Print Active/Inactive counts


Updated dataset saved as C:\Users\ngaga\OneDrive\Desktop\aragen\AChE_ligands_weighted_activity.csv
Activity
Inactive    3819
Active      3730
Name: count, dtype: int64


# with xgboost

In [21]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

# Load dataset
file_path = r"C:\Users\ngaga\OneDrive\Desktop\aragen\AChE_ligands_weighted_activity.csv"
df = pd.read_csv(file_path)

# Ensure SMILES column is clean and contains valid strings
df['SMILES'] = df['SMILES'].replace(np.nan, '').replace('nan', '').astype(str)

# Convert SMILES to Morgan Fingerprints (2048-bit vector)
def smiles_to_fingerprint(smiles):
    if not smiles or smiles.strip() == '':  # Handle empty SMILES
        return [0] * 2048  # Return a zero vector

    mol = Chem.MolFromSmiles(smiles)
    if mol:
        return list(AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048))
    else:
        return [0] * 2048  # Return zero vector if conversion fails

# Generate fingerprints
df['Fingerprint'] = df['SMILES'].apply(smiles_to_fingerprint)

# Convert fingerprint list into separate columns (feature matrix)
X = np.array(df['Fingerprint'].tolist())

# Target variable (Active = 1, Inactive = 0)
y = (df['Activity'] == 'Active').astype(int)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Train XGBoost model
model = XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluation
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print(classification_report(y_test, y_pred))

# Save model
joblib.dump(model, "ache_ligand_classifier.pkl")
print("Model saved as ache_ligand_classifier.pkl")


Accuracy: 0.8397
              precision    recall  f1-score   support

           0       0.82      0.87      0.85       764
           1       0.86      0.80      0.83       746

    accuracy                           0.84      1510
   macro avg       0.84      0.84      0.84      1510
weighted avg       0.84      0.84      0.84      1510

Model saved as ache_ligand_classifier.pkl


# multiple models testing for better accuracy

In [24]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load dataset
file_path = r"C:\Users\ngaga\OneDrive\Desktop\aragen\AChE_ligands_weighted_activity.csv"
df = pd.read_csv(file_path)

# Convert SMILES to Morgan Fingerprints (2048-bit vector)
def smiles_to_fingerprint(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        return list(AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048))
    else:
        return [0] * 2048  # Return zero vector if conversion fails

# Drop NaN SMILES values
df = df.dropna(subset=['SMILES'])

# Generate fingerprints
df['Fingerprint'] = df['SMILES'].apply(smiles_to_fingerprint)

# Convert fingerprint list into separate columns (feature matrix)
X = np.array(df['Fingerprint'].tolist())

# Target variable (Active = 1, Inactive = 0)
y = (df['Activity'] == 'Active').astype(int)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Define models
models = {
    "Logistic Regression": LogisticRegression(max_iter=2000),
    "Random Forest": RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42),
    "XGBoost": XGBClassifier(n_estimators=200, max_depth=7, learning_rate=0.05, random_state=42),
    "LightGBM": LGBMClassifier(n_estimators=200, max_depth=7, learning_rate=0.05, random_state=42),
    "SVM": SVC(kernel='rbf', probability=True),
    "MLP (Neural Network)": MLPClassifier(hidden_layer_sizes=(512, 256), max_iter=300, random_state=42)
}

# Train and evaluate models
results = {}
for name, model in models.items():
    print(f"\nTraining {name}...")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy: {acc:.4f}")
    
    results[name] = acc

# Print best model
best_model = max(results, key=results.get)
print(f"\nBest model: {best_model} with accuracy: {results[best_model]:.4f}")



Training Logistic Regression...
Logistic Regression Accuracy: 0.8489

Training Random Forest...
Random Forest Accuracy: 0.7939

Training XGBoost...
XGBoost Accuracy: 0.8588

Training LightGBM...


C:\Users\ngaga\AppData\Roaming\Python\Python311\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\ngaga\AppData\Roaming\Python\Python311\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Program Files\Python311\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\Python311\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Program Files\Python311\Lib\subprocess.py", line 1538, in _execute_ch

[LightGBM] [Info] Number of positive: 2980, number of negative: 3055
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.104649 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2912
[LightGBM] [Info] Number of data points in the train set: 6035, number of used features: 1456
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.493786 -> initscore=-0.024856
[LightGBM] [Info] Start training from score -0.024856
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

C:\Users\ngaga\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


SVM Accuracy: 0.8549

Training MLP (Neural Network)...
MLP (Neural Network) Accuracy: 0.8569

Best model: XGBoost with accuracy: 0.8588


# classification prediction using pretrained model chemberta

In [26]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Load dataset
file_path = r"C:\Users\ngaga\OneDrive\Desktop\aragen\AChE_ligands_weighted_activity.csv"
df = pd.read_csv(file_path)

# Drop NaNs
df = df.dropna(subset=['SMILES', 'Activity'])

# Convert labels to binary (Active = 1, Inactive = 0)
df['Activity'] = df['Activity'].map({'Active': 1, 'Inactive': 0})

# Train-test split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['SMILES'].tolist(), df['Activity'].tolist(), test_size=0.2, random_state=42, stratify=df['Activity']
)

# Load ChemBERTa tokenizer
tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-77M-MTR")

# Create Dataset class
class SmilesDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(self.texts[idx], truncation=True, padding="max_length", max_length=128, return_tensors="pt")
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# Create PyTorch datasets
train_dataset = SmilesDataset(train_texts, train_labels, tokenizer)
test_dataset = SmilesDataset(test_texts, test_labels, tokenizer)

# Load pre-trained ChemBERTa model for classification
model = AutoModelForSequenceClassification.from_pretrained("DeepChem/ChemBERTa-77M-MTR", num_labels=2)

# Training arguments
training_args = TrainingArguments(
    output_dir="./chemberta_results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=2,
    learning_rate=2e-5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

# Define evaluation metric
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = torch.argmax(torch.tensor(logits), dim=-1).numpy()
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Save the fine-tuned model
trainer.save_model("./fine_tuned_chemberta")
tokenizer.save_pretrained("./fine_tuned_chemberta")
print("Fine-tuned ChemBERTa model saved!")


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MTR and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\ngaga\AppData\Roaming\Python\Python311\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.612600,0.589709,0.681909,0.681259,0.668456,0.674797
2,0.548800,0.537832,0.722333,0.724518,0.706040,0.715160
3,0.490300,0.511300,0.751491,0.747989,0.748993,0.748491
4,0.453400,0.503730,0.763419,0.753927,0.773154,0.763419
5,0.484200,0.500911,0.767396,0.748737,0.795973,0.771633
6,0.451300,0.490734,0.781312,0.769831,0.794631,0.782034
7,0.438400,0.490858,0.785951,0.768448,0.810738,0.789027
8,0.465200,0.488821,0.787939,0.767296,0.818792,0.792208
9,0.435400,0.485204,0.790590,0.772554,0.816107,0.793734
10,0.455200,0.486388,0.789927,0.770202,0.818792,0.793754


Fine-tuned ChemBERTa model saved!


# IT CAN BE SEEN THAT XGBOOST HAS BEEN PERFORMING WELL COMPARED TO OTHERS BEFORE FEATURE EXTRACTION

## feature extraction (descriptors)

In [29]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors

# Load dataset
df = pd.read_csv(r"C:\Users\ngaga\OneDrive\Desktop\aragen\AChE_ligands_weighted_activity.csv")  # Replace with actual dataset path

# Ensure SMILES column is of string type and handle missing values
df["SMILES"] = df["SMILES"].astype(str).str.strip()  # Convert to string & remove spaces
df = df.dropna(subset=["SMILES"])  # Drop rows where SMILES is NaN

# Function to compute descriptors
def compute_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return [np.nan] * 6  # Return NaN if invalid SMILES

    return [
        Descriptors.MolWt(mol),                        # Molecular Weight
        Descriptors.MolLogP(mol),                      # LogP
        Descriptors.TPSA(mol),                         # Topological Polar Surface Area
        rdMolDescriptors.CalcNumHBA(mol),              # Hydrogen Bond Acceptors
        rdMolDescriptors.CalcNumHBD(mol),              # Hydrogen Bond Donors
        rdMolDescriptors.CalcNumRotatableBonds(mol),   # Rotatable Bonds
    ]

# Apply descriptor calculation safely
df[["MolWt", "LogP", "TPSA", "HBA", "HBD", "RotBonds"]] = df["SMILES"].apply(
    lambda x: pd.Series(compute_descriptors(x) if isinstance(x, str) else [np.nan] * 6)
)

# Save the updated dataset
df.to_csv("dataset_with_descriptors.csv", index=False)

print(" Descriptors added and dataset saved!")


[14:15:10] SMILES Parse Error: syntax error while parsing: nan
[14:15:10] SMILES Parse Error: Failed parsing SMILES 'nan' for input: 'nan'
[14:15:15] SMILES Parse Error: syntax error while parsing: nan
[14:15:15] SMILES Parse Error: Failed parsing SMILES 'nan' for input: 'nan'
[14:15:15] SMILES Parse Error: syntax error while parsing: nan
[14:15:15] SMILES Parse Error: Failed parsing SMILES 'nan' for input: 'nan'
[14:15:15] SMILES Parse Error: syntax error while parsing: nan
[14:15:15] SMILES Parse Error: Failed parsing SMILES 'nan' for input: 'nan'
[14:15:15] SMILES Parse Error: syntax error while parsing: nan
[14:15:15] SMILES Parse Error: Failed parsing SMILES 'nan' for input: 'nan'


✅ Descriptors added and dataset saved!


# training multiple ML models with feature extracted dataset to find the best model before normalization

In [32]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Load dataset
df = pd.read_csv(R"C:\Users\ngaga\OneDrive\Desktop\aragen\AChE_dataset\dataset_with_descriptors.csv")

# Drop any missing values
df.dropna(inplace=True)

# Convert Activity column to binary (Active = 1, Inactive = 0)
df["Activity"] = df["Activity"].map({"Active": 1, "Inactive": 0})

# Define feature columns (excluding non-numeric columns)
feature_cols = ["MolWt", "LogP", "TPSA", "HBA", "HBD", "RotBonds"]
X = df[feature_cols]  # Features
y = df["Activity"]  # Target

# Split into training and testing sets (80-20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Standardize features (important for models like SVM & Logistic Regression)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define multiple ML models
models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "Gradient Boosting": GradientBoostingClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "SVM": SVC(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss"),
    "LightGBM": LGBMClassifier(),
    "CatBoost": CatBoostClassifier(verbose=0)
}

# Train and evaluate models
results = []

for name, model in models.items():
    print(f"🔹 Training {name}...")
    
    model.fit(X_train, y_train)  # Train model
    y_pred = model.predict(X_test)  # Predict on test set
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    results.append({"Model": name, "Accuracy": accuracy, "Precision": precision, "Recall": recall, "F1-Score": f1})

# Convert results to DataFrame
results_df = pd.DataFrame(results).sort_values(by="F1-Score", ascending=False)

# Display results
print("\n Model Performance Comparison:")
print(results_df)

# Save results to CSV
results_df.to_csv("model_comparison_results.csv", index=False)

print("\n Best model based on F1-score:", results_df.iloc[0]["Model"])


🔹 Training Logistic Regression...
🔹 Training Random Forest...
🔹 Training Gradient Boosting...
🔹 Training Decision Tree...
🔹 Training SVM...
🔹 Training K-Nearest Neighbors...
🔹 Training XGBoost...
🔹 Training LightGBM...
[LightGBM] [Info] Number of positive: 2980, number of negative: 3055
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000209 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 827
[LightGBM] [Info] Number of data points in the train set: 6035, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.493786 -> initscore=-0.024856
[LightGBM] [Info] Start training from score -0.024856


C:\Users\ngaga\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:158: UserWarning: [14:28:48] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\ngaga\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


🔹 Training CatBoost...

 Model Performance Comparison:
                 Model  Accuracy  Precision    Recall  F1-Score
1        Random Forest  0.804506   0.804054  0.798658  0.801347
6              XGBoost  0.781312   0.784636  0.767785  0.776119
5  K-Nearest Neighbors  0.773360   0.758665  0.793289  0.775591
7             LightGBM  0.778661   0.785814  0.758389  0.771858
8             CatBoost  0.768721   0.774238  0.750336  0.762100
3        Decision Tree  0.752816   0.742820  0.763758  0.753144
2    Gradient Boosting  0.707091   0.713681  0.679195  0.696011
4                  SVM  0.672631   0.666667  0.673826  0.670227
0  Logistic Regression  0.582505   0.587253  0.519463  0.551282

 Best model based on F1-score: Random Forest


# finetuning chemberta model to get higher accuracy

In [34]:
# Required Libraries
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

# Set device (CPU only)
device = torch.device("cpu")

# Load dataset
df = pd.read_csv(r"C:\Users\ngaga\OneDrive\Desktop\aragen\AChE_dataset\dataset_with_descriptors.csv")

# Drop missing values
df.dropna(inplace=True)

# Define input features (including SMILES and molecular descriptors)
feature_columns = ["MolWt", "LogP", "TPSA", "HBA", "HBD", "RotBonds"]  # Molecular descriptors
X_features = df[feature_columns].values  # Extract numerical features

# Normalize features
scaler = StandardScaler()
X_features = scaler.fit_transform(X_features)

# Encode Activity label (0 = Inactive, 1 = Active)
df["Activity"] = df["Activity"].map({"Inactive": 0, "Active": 1})
y = df["Activity"].values  # Target variable

# Train-test split (80-20)
X_train, X_test, y_train, y_test, train_features, test_features = train_test_split(
    df["SMILES"], y, X_features, test_size=0.2, random_state=42, stratify=y
)

# Load ChemBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained("seyonec/ChemBERTa-zinc-base-v1")

# Define dataset class
class ChemDataset(Dataset):
    def __init__(self, smiles_list, features, labels, tokenizer, max_length=128):
        self.smiles_list = smiles_list
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.smiles_list)

    def __getitem__(self, idx):
        smiles = self.smiles_list.iloc[idx]
        label = self.labels[idx]
        feature = self.features[idx]

        # Tokenize SMILES
        encoding = self.tokenizer(
            smiles,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt",
        )

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "features": feature,
            "labels": label,
        }

# Create datasets
train_dataset = ChemDataset(X_train, train_features, y_train, tokenizer)
test_dataset = ChemDataset(X_test, test_features, y_test, tokenizer)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print("✅ Dataset prepared successfully!")

# Define ChemBERTa-based model with numerical feature fusion
class ChemBERTaWithFeatures(nn.Module):
    def __init__(self, bert_model_name="seyonec/ChemBERTa-zinc-base-v1", feature_dim=6, num_classes=2):
        super(ChemBERTaWithFeatures, self).__init__()
        self.chemberta = RobertaModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.3)
        self.feature_fc = nn.Linear(feature_dim, 64)  # Project numerical features
        self.classifier = nn.Linear(self.chemberta.config.hidden_size + 64, num_classes)

    def forward(self, input_ids, attention_mask, features):
        bert_output = self.chemberta(input_ids=input_ids, attention_mask=attention_mask)
        chemberta_embedding = bert_output.pooler_output  # Get pooled embedding

        # Process numerical features
        feature_output = torch.relu(self.feature_fc(features))

        # Concatenate ChemBERTa embedding & extracted features
        combined = torch.cat((chemberta_embedding, feature_output), dim=1)

        # Final classification layer
        output = self.classifier(self.dropout(combined))

        return output

# Initialize model
model = ChemBERTaWithFeatures()

print("✅ Model loaded successfully!")

# Training setup
epochs = 5
learning_rate = 2e-5
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        features = batch["features"]
        labels = batch["labels"]

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask, features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss / len(train_loader):.4f}")

print("✅ Training complete!")

# Evaluation function
def evaluate(model, dataloader):
    model.eval()
    preds, true_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"]
            attention_mask = batch["attention_mask"]
            features = batch["features"]
            labels = batch["labels"]

            outputs = model(input_ids, attention_mask, features)
            predictions = torch.argmax(outputs, dim=1).numpy()

            preds.extend(predictions)
            true_labels.extend(labels.numpy())

    acc = accuracy_score(true_labels, preds)
    prec = precision_score(true_labels, preds)
    rec = recall_score(true_labels, preds)
    f1 = f1_score(true_labels, preds)

    print(f"✅ Accuracy: {acc:.4f} | Precision: {prec:.4f} | Recall: {rec:.4f} | F1 Score: {f1:.4f}")

# Evaluate on test set
evaluate(model, test_loader)


✅ Dataset prepared successfully!
✅ Model loaded successfully!


100%|████████████████████████████████████████████████████████████████████████████████| 189/189 [19:11<00:00,  6.09s/it]


Epoch 1/5 - Loss: 0.5617


100%|████████████████████████████████████████████████████████████████████████████████| 189/189 [20:51<00:00,  6.62s/it]


Epoch 2/5 - Loss: 0.4223


...

100%|████████████████████████████████████████████████████████████████████████████████| 189/189 [32:18<00:00, 10.26s/it]


Epoch 4/5 - Loss: 0.3023


100%|████████████████████████████████████████████████████████████████████████████████| 189/189 [16:53<00:00,  5.36s/it]

Epoch 5/5 - Loss: 0.2677
✅ Training complete!


✅ Accuracy: 0.8376 | Precision: 0.8238 | Recall: 0.8537 | F1 Score: 0.8385


# XGBOOST model using just smiles

In [36]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier

# Load dataset
file_path = r"C:\Users\ngaga\OneDrive\Desktop\aragen\AChE_dataset\AChE_ligands_weighted_activity.csv"
df = pd.read_csv(file_path)

# Convert SMILES to Morgan Fingerprints (2048-bit vector)
def smiles_to_fingerprint(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        return list(AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048))
    else:
        return [0] * 2048  # Return zero vector if conversion fails

# Drop NaN SMILES values
df = df.dropna(subset=['SMILES'])

# Generate fingerprints
df['Fingerprint'] = df['SMILES'].apply(smiles_to_fingerprint)

# Convert fingerprint list into separate columns (feature matrix)
X = np.array(df['Fingerprint'].tolist())

# Target variable (Active = 1, Inactive = 0)
y = (df['Activity'] == 'Active').astype(int)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Initialize XGBoost model
xgb_model = XGBClassifier(n_estimators=200, max_depth=7, learning_rate=0.05, random_state=42)

# Train the model
print("\nTraining XGBoost...")
xgb_model.fit(X_train, y_train)

# Predict on test set
y_pred = xgb_model.predict(X_test)

# Evaluate performance
acc = accuracy_score(y_test, y_pred)
print(f"\nXGBoost Accuracy: {acc:.4f}")

# Print detailed classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))



Training XGBoost...

XGBoost Accuracy: 0.8588

Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.88      0.86       764
           1       0.87      0.84      0.85       745

    accuracy                           0.86      1509
   macro avg       0.86      0.86      0.86      1509
weighted avg       0.86      0.86      0.86      1509



# XGBOOST using smiles and molecular descriptors

In [38]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier

# Load dataset
file_path = r"C:\Users\ngaga\OneDrive\Desktop\aragen\AChE_dataset\dataset_with_descriptors.csv"
df = pd.read_csv(file_path)

# Drop missing values
df.dropna(subset=['SMILES'], inplace=True)  

# Convert SMILES to Morgan Fingerprints (2048-bit vector)
def smiles_to_fingerprint(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        return list(AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048))
    else:
        return [0] * 2048  # Return zero vector if conversion fails

# Generate fingerprints
df['Fingerprint'] = df['SMILES'].apply(smiles_to_fingerprint)

# Extract molecular descriptors as features
descriptor_columns = ["MolWt", "LogP", "TPSA", "HBA", "HBD", "RotBonds"]
X_descriptors = df[descriptor_columns].values  # Convert to NumPy array

# Convert fingerprint lists into NumPy array
X_fingerprints = np.array(df['Fingerprint'].tolist())

# Concatenate fingerprints and molecular descriptors
X = np.hstack((X_fingerprints, X_descriptors))  # Combine both feature sets

# Target variable (Active = 1, Inactive = 0)
y = (df["Activity"] == "Active").astype(int)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Initialize XGBoost model
xgb_model = XGBClassifier(n_estimators=200, max_depth=7, learning_rate=0.05, random_state=42)

# Train the model
print("\nTraining XGBoost on SMILES + Molecular Descriptors...")
xgb_model.fit(X_train, y_train)

# Predict on test set
y_pred = xgb_model.predict(X_test)

# Evaluate performance
acc = accuracy_score(y_test, y_pred)
print(f"\nXGBoost Accuracy: {acc:.4f}")

# Print detailed classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))



Training XGBoost on SMILES + Molecular Descriptors...

XGBoost Accuracy: 0.8588

Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.88      0.86       764
           1       0.88      0.83      0.85       745

    accuracy                           0.86      1509
   macro avg       0.86      0.86      0.86      1509
weighted avg       0.86      0.86      0.86      1509



# XGBOOST after normalizing the molecular descriptors

In [62]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier

# Load dataset
file_path = r"C:\Users\ngaga\OneDrive\Desktop\aragen\AChE_dataset\dataset_with_descriptors.csv"
df = pd.read_csv(file_path)

# Drop missing values
df.dropna(subset=['SMILES'], inplace=True)  

# Convert SMILES to Morgan Fingerprints (2048-bit vector)
def smiles_to_fingerprint(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        return list(AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048))
    else:
        return [0] * 2048  # Return zero vector if conversion fails

# Generate fingerprints
df['Fingerprint'] = df['SMILES'].apply(smiles_to_fingerprint)

# Extract molecular descriptors as features
descriptor_columns = ["MolWt", "LogP", "TPSA", "HBA", "HBD", "RotBonds"]
X_descriptors = df[descriptor_columns].values  # Convert to NumPy array

# Normalize molecular descriptors
scaler = StandardScaler()
X_descriptors_scaled = scaler.fit_transform(X_descriptors)

# Convert fingerprint lists into NumPy array
X_fingerprints = np.array(df['Fingerprint'].tolist())

# Concatenate fingerprints and normalized molecular descriptors
X = np.hstack((X_fingerprints, X_descriptors_scaled))  # Combine both feature sets

# Target variable (Active = 1, Inactive = 0)
y = (df["Activity"] == "Active").astype(int)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Initialize XGBoost model
xgb_model = XGBClassifier(n_estimators=200, max_depth=7, learning_rate=0.05, random_state=42)

# Train the model
print("\nTraining XGBoost on SMILES + Normalized Molecular Descriptors...")
xgb_model.fit(X_train, y_train)

# Predict on test set
y_pred = xgb_model.predict(X_test)

# Evaluate performance
acc = accuracy_score(y_test, y_pred)
print(f"\nXGBoost Accuracy: {acc:.4f}")

# Print detailed classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))



Training XGBoost on SMILES + Normalized Molecular Descriptors...

XGBoost Accuracy: 0.8588

Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.88      0.86       764
           1       0.88      0.83      0.85       745

    accuracy                           0.86      1509
   macro avg       0.86      0.86      0.86      1509
weighted avg       0.86      0.86      0.86      1509



### we can see that even after adding molecular descriptors the model is performing the same because the model can grasp all of those features from the smiles itself. Hence proceeding with the smiles dataset instead of depending on molecular descriptors

# predicting random smile if its active or inactive

In [57]:
import joblib
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem

# Load the trained model
loaded_model = joblib.load("xgboost_AChE_model.pkl")

# Function to predict activity from a SMILES string
def predict_activity(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        fingerprint = list(AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048))
        fingerprint = np.array(fingerprint).reshape(1, -1)  # Reshape for prediction
        prediction = loaded_model.predict(fingerprint)
        return "Active" if prediction[0] == 1 else "Inactive"
    else:
        return "Invalid SMILES"

# Example usage
smiles_input = "NC(=O)c1ccc(CNc2ccc(OCC3CCCCC3)cc2)cc1"
print(f"Prediction for {smiles_input}: {predict_activity(smiles_input)}")


Prediction for NC(=O)c1ccc(CNc2ccc(OCC3CCCCC3)cc2)cc1: Inactive


# prediction for smiles with a txt file with confidence score

In [61]:
import joblib
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem

# Load the trained model
loaded_model = joblib.load("xgboost_AChE_model.pkl")

# Function to predict activity and confidence score from a SMILES string
def predict_activity(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        fingerprint = list(AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048))
        fingerprint = np.array(fingerprint).reshape(1, -1)  # Reshape for prediction
        
        # Get prediction and confidence score
        prediction = loaded_model.predict(fingerprint)[0]
        confidence = loaded_model.predict_proba(fingerprint)[0][prediction]  # Probability of the predicted class
        
        activity = "Active" if prediction == 1 else "Inactive"
        return activity, confidence
    else:
        return "Invalid SMILES", None

# Function to process a file with multiple SMILES and display results
def process_smiles_file(input_file):
    with open(input_file, 'r') as f:
        smiles_list = [line.strip() for line in f.readlines()]  # Read all SMILES from file

    print("\nPredictions:\n")
    print(f"{'SMILES':<50} {'Activity':<10} {'Confidence':<10}")
    print("-" * 75)

    for smiles in smiles_list:
        activity, confidence = predict_activity(smiles)
        if confidence is not None:
            print(f"{smiles:<50} {activity:<10} {confidence:.2f}")
        else:
            print(f"{smiles:<50} {'Invalid':<10} {'N/A':<10}")

# Example usage
input_file = r"C:\Users\ngaga\Downloads\valid_molecules.txt"  # Input file with multiple SMILES (one per line)
process_smiles_file(input_file)



Predictions:

SMILES                                             Activity   Confidence
---------------------------------------------------------------------------
COc1ccc2c(c1)CCC(C1CCN(c3ccccc3)CC1)C2             Inactive   0.52
COc1cc2c(cc1OC)C(=O)C(CC1CCN(Cc3ccccc3)CC1)C2      Active     0.91
O=C(NCC1CCN(Cc2ccccc2)CC1)c1cc(=O)c2cc(O)ccc2o1    Inactive   0.69
O=C(CCc1ccccc1)NCC1CCN(Cc2ccccc2)CC1               Inactive   0.65
COc1cc2cc(-c3ccc(CN(C)Cc4ccccc4)cc3)c(=O)oc2cc1OC  Active     0.81
COc1cc2c(cc1OC)C(=O)/C(=C/c1ccc(N3CCCC3)cc1)C2     Active     0.70
CCNCC(=O)Nc1ccc2nc3n(c(=O)c2c1)CCC3                Active     0.82
Nc1c2c(nc3ccccc13)CCCC2                            Active     0.92
COc1cc2c(cc1OC)C(=O)/C(=C/c1cc[n+](Cc3cccc(Cl)c3)cc1C(C)=O)C2.[Br-] Active     0.85
CNc1c2c(nc3ccccc13)CCCC2                           Active     0.77
Nc1c2c(nc3cc(Cl)ccc13)CCCC2                        Active     0.93
Cc1ccc2nc(C(=O)NCC3CCN(Cc4ccccc4)CC3)ccc2c1        Inactive   0.69
COc1cc2c(cc1OC)